In [11]:
import os
import pandas as pd
import numpy as np
import re

In [2]:
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10

In [3]:
#loading in glove
word2vec = {}
with open(os.path.join('D:\\GLOVE\\glove.6B.100d.txt'), encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
print('Found {} word vectors.'.format(len(word2vec)))

Found 400000 word vectors.


In [4]:
file_directory = 'D://Data//Kaggle//disaster_tweets//'
train_file = os.path.join(file_directory, 'train.csv')
test_file = os.path.join(file_directory, 'test.csv')

df = pd.read_csv(train_file)
df_test = pd.read_csv(test_file)

In [16]:
class df_preprocess:
    def __init__ (self, df):
        self.df = df
    
    def text_cleaner (self, text):
        text = text.lower()
        text = re.sub(r"(?:\@|https?\://)\S+", "", text)
        return text
    
    def clean_column(self, col):
        self.df[col] = self.df[col].apply(lambda x: self.text_cleaner(x))
        return self.df

In [17]:
pp = df_preprocess(df)
pp.clean_column('text')

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this #earthquake m...,1
1,4,NaN,NaN,forest fire near la ronge sask. canada,1
2,5,NaN,NaN,all residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,just got sent this photo from ruby #alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,the out of control wild fires in california ...,1
7610,10871,NaN,NaN,m1.94 [01:04 utc]?5km s of volcano hawaii.,1
7611,10872,NaN,NaN,police investigating after an e-bike collided ...,1


In [9]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
